# Pandas — Joins, Edge Cases & Data Validation



In [1]:
# Initialize

#Imports
import pandas as pd
pd.set_option("display.max_columns", None)
# Load Data
employees = pd.read_csv("../data/employees.csv")
departments = pd.read_csv("../data/departments.csv")
sales = pd.read_csv("../data/sales.csv")

In [2]:
# INNER JOIN
# Keeps only records where emp_id exists in both tables

inner_join_df = sales.merge(
    employees,
    on="emp_id",
    how="inner"
)

print("Inner Join Shape:", inner_join_df.shape)
inner_join_df.head()

Inner Join Shape: (20, 9)


,sale_id,emp_id,amount,sale_date,region,name,department,salary,joining_date
0,1,1,12000,2023-01-10,North,Amit,Engineering,80000,2021-06-15
1,4,1,7000,2023-02-05,North,Amit,Engineering,80000,2021-06-15
2,15,1,10000,2023-05-05,North,Amit,Engineering,80000,2021-06-15
3,2,2,15000,2023-01-12,East,Neha,Engineering,75000,2022-03-10
4,14,2,15500,2023-05-01,East,Neha,Engineering,75000,2022-03-10


In [4]:
# LEFT JOIN
# Keeps all sales even if employee is missing

left_join_df = sales.merge(
    employees,
    on="emp_id",
    how="left"
)

print("Left Join Shape:", left_join_df.shape)
left_join_df.head()

Left Join Shape: (20, 9)


,sale_id,emp_id,amount,sale_date,region,name,department,salary,joining_date
0,1,1,12000,2023-01-10,North,Amit,Engineering,80000,2021-06-15
1,2,2,15000,2023-01-12,East,Neha,Engineering,75000,2022-03-10
2,3,5,8000,2023-02-01,West,Karan,Sales,60000,2023-02-01
3,4,1,7000,2023-02-05,North,Amit,Engineering,80000,2021-06-15
4,5,3,4000,2023-02-10,South,Ravi,HR,50000,2020-01-20


In [5]:
# Detect sales records that have no matching employee

invalid_sales = left_join_df[left_join_df["name"].isna()]

invalid_sales

,sale_id,emp_id,amount,sale_date,region,name,department,salary,joining_date


In [6]:
# Employees who have not made any sales

sales_emp_ids = set(sales["emp_id"])

employees_no_sales = employees[~employees["emp_id"].isin(sales_emp_ids)]

employees_no_sales

,emp_id,name,department,salary,joining_date
3,4,Pooja,HR,55000,2021-11-05
7,8,Anita,Finance,70000,2021-04-01
9,10,Meena,HR,52000,2022-06-20
13,14,Kavita,HR,54000,2019-02-14
15,16,Deepak,Finance,75000,2022-09-09
17,18,Manoj,HR,51000,2020-10-10


In [7]:
# Joining employees with departments
# This simulates dimension normalization

employees_with_dept_id = employees.merge(
    departments,
    left_on="department",
    right_on="dept_name",
    how="left"
)

employees_with_dept_id.head()

,emp_id,name,department,salary,joining_date,dept_id,dept_name
0,1,Amit,Engineering,80000,2021-06-15,101,Engineering
1,2,Neha,Engineering,75000,2022-03-10,101,Engineering
2,3,Ravi,HR,50000,2020-01-20,102,HR
3,4,Pooja,HR,55000,2021-11-05,102,HR
4,5,Karan,Sales,60000,2023-02-01,103,Sales


In [8]:
# Enrich sales with employee and department information

sales_enriched = (
    sales
        .merge(employees, on="emp_id", how="left")
        .merge(departments, left_on="department", right_on="dept_name", how="left")
)

sales_enriched.head()

,sale_id,emp_id,amount,sale_date,region,name,department,salary,joining_date,dept_id,dept_name
0,1,1,12000,2023-01-10,North,Amit,Engineering,80000,2021-06-15,101,Engineering
1,2,2,15000,2023-01-12,East,Neha,Engineering,75000,2022-03-10,101,Engineering
2,3,5,8000,2023-02-01,West,Karan,Sales,60000,2023-02-01,103,Sales
3,4,1,7000,2023-02-05,North,Amit,Engineering,80000,2021-06-15,101,Engineering
4,5,3,4000,2023-02-10,South,Ravi,HR,50000,2020-01-20,102,HR


In [10]:
# Simulating duplicate employee record
employees_duplicate = pd.concat([employees, employees.iloc[[0]]])

print("Original employees:", employees.shape)
print("After duplication:", employees_duplicate.shape)

exploded_join = sales.merge(
    employees_duplicate,
    on="emp_id",
    how="left"
)

print("Original sales:", sales.shape)
print("After join:", exploded_join.shape)

Original employees: (20, 5)
After duplication: (21, 5)
Original sales: (20, 5)
After join: (23, 9)


In [ ]:
# Checking if emp_id is unique

duplicate_emp_ids = employees["department"].duplicated().any()
print(duplicate_emp_ids)
print("Is emp_id unique?", not duplicate_emp_ids)

0     False
1      True
2     False
3      True
4     False
5      True
6      True
7     False
8      True
9      True
10     True
11     True
12     True
13     True
14     True
15     True
16     True
17     True
18     True
19     True
Name: department, dtype: bool


ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().